# Vehicle Detection Project

The goals / steps of this project are the following:

* Perform a Histogram of Oriented Gradients (HOG) feature extraction on a labeled training set of images and train a classifier Linear SVM classifier
* Optionally, you can also apply a color transform and append binned color features, as well as histograms of color, to your HOG feature vector. 
* Note: for those first two steps don't forget to normalize your features and randomize a selection for training and testing.
* Implement a sliding-window technique and use your trained classifier to search for vehicles in images.
* Run your pipeline on a video stream (start with the test_video.mp4 and later implement on full project_video.mp4) and create a heat map of recurring detections frame by frame to reject outliers and follow detected vehicles.
* Estimate a bounding box for vehicles detected.

[//]: # (Image References)
[image1]: ./writeup/car_not_car.png
[image2]: ./writeup/hog.png
[image3]: ./writeup/sliding_windows.png
[image4]: ./writeup/windowed.png
[image5]: ./writeup/bboxes_and_heat.png
[image6]: ./writeup/labels_map.png
[image7]: ./writeup/output_bboxes.png
[video1]: ./writeup/project_video.mp4

### Here I will consider the rubric points individually and describe how I addressed each point in my implementation.  

---
### Writeup / README

#### 1. Provide a Writeup / README that includes all the rubric points and how you addressed each one.
This is the writeup. The python3 code to process the images is in `find_car.py`

### Histogram of Oriented Gradients (HOG)
#### 1.  How I extracted HOG features from the training images.
 
I started by reading in all the `vehicle` and `non-vehicle` .png image names supplied in the `training_data` folder.  Here is an example of one of each of the `vehicle` and `non-vehicle` classes:

![alt text][image1]


The code for this step is in a function `car_notcar_image_names ()` or in lines #52 through #74 of the file called `find_car.py`). 

* To be consistent `matplotlib.image as mpimg` is used everywhere to read in the image files. The important caveat is the training dataset provided for this project ( vehicle and non-vehicle images) are in the .png format and matplotlib image will read these in on a scale of 0 to 1. Additionally test images and video frames are jpeg and matplotlib image will read .jpg images in on a scale of 0 to 255.

* Hence the histogram for color features bin_range is (0,1) as in: `line 108: def color_hist(img, nbins=32, bins_range=(0, 1))`

* Jpeg images are normalised 0-1 as in: `line 479: image = orig_image.astype(np.float32)/255`


I then explored different color spaces and different `skimage.feature.hog()` parameters (`orientations`, `pixels_per_cell`, and `cells_per_block`).  I grabbed random images from each of the two classes and displayed them to get a feel for what the `skimage.feature.hog()` output looks like.

Here is an example using the `YCrCb` color space and HOG parameters of `orientations=8`, `pixels_per_cell=(8, 8)` and `cells_per_block=(2, 2)`:

![alt text][image2]

#### 2. How I settled on your final choice of HOG parameters.
From the HOG image previews it was evident that the hog for each color channel looked the same. So choosing any one or ALL of them would not make a difference. Since the images were 64x64 picking a 8x8 pixels per cell seemed reasonable. Also I though 8 orientations was sufficient. The function `get_hog_features` is where this is performed.


#### 3. How I trained a classifier using your selected HOG features (and color features if you used them).

The function `single_img_features` gets an array of all the festure vectors HOG, spatial and color_hist. `extract_features_all_imagenames` assembles the features vector for all the images of a particular set.
2 Sets of features were extracted car and not_car to make up the input feature vector. 80% of the data (after random shuffling was used in the `train_data_set_and_generate_model` function where a linear SVM classifier was used.
`svc = LinearSVC()`. After training the model parameters was dumped to a pickle file to be reused repeatedly in testing.


### Sliding Window Search

#### 1.The following sizes and start_stop was used to generate the sliding windows to search for the car

`
sliding_window_params = 

{'x_start_stop': [600,1100], 'y_start_stop': [384,496], 'xy_overlap':(0.8,0.8), 'xy_window':(80,80), 'color':COLOR_BLUE},
{'x_start_stop': [600,1180], 'y_start_stop': [384,554], 'xy_overlap':(0.8, 0.8),'xy_window':(128, 128),'color': COLOR_GREEN},
{'x_start_stop': [580,1280], 'y_start_stop': [440,640], 'xy_overlap':(0.8, 0.8),  'xy_window':(160, 160),'color': COLOR_ORANGE},
{'x_start_stop': [520, 1280], 'y_start_stop': [460, 708], 'xy_overlap': (0.8, 0.8), 'xy_window': (192, 192), 'color': COLOR_PURPLE}

`

Initially `x_start_stop` was set to (None,None). This detects a the cars on the other lanes of the freeway as well. In order to restrict the search to the driving side the `x_start_stop` was tightened to start from about half-way mark in the x direction. The sliding windows are plotted in the image below.

![alt text][image3]


#### 2. An example test image to demonstrate that the pipeline is working. 


Ultimately I searched using the following parameters, which provided a nice result.

`   color_space = 'YCrCb'  # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
    orient = 9  # HOG orientations
    pix_per_cell = 8  # HOG pixels per cell
    cell_per_block = 2  # HOG cells per block
    hog_channel = "ALL"  # Can be 0, 1, 2, or "ALL"
    spatial_size = (16, 16)  # Spatial binning dimensions
    hist_bins = 32  # Number of histogram bins
    spatial_feat = True  # Spatial features on or off
    hist_feat = True  # Histogram features on or off
    hog_feat = True  # HOG features on or off
`


Here's an example result showing the heatmap on a test_image, the result of `scipy.ndimage.measurements.label()` and the bounding boxes then overlaid on the last frame:
![alt text][image4]
---


### Video Implementation


#### 1.  The pipeline was run on the video and the output in the ./test_videos_output/project_video.mp4
Here's a [link to my video result](./test_videos_output/project_video.mp4)


#### 2. Describe how (and identify where in your code) you implemented some kind of filter for false positives and some method for combining overlapping bounding boxes.

The function `process_jpeg_image` does the multi_window heat_map and thresholding. From which I recorded the positions of positive detections in each frame of the video.  From the positive detections I created a heatmap accumulator and then thresholded that map to identify vehicle positions (See below).  I then used `scipy.ndimage.measurements.label()` to identify individual blobs in the heatmap.  I then assumed each blob corresponded to a vehicle.  I constructed bounding boxes to cover the area of each blob detected.  

`
import numpy as np

#class to maintain a list and accumulation of N recent heatmaps
#A threshold can also be applied to the accumulated heatmap
#i.e. heatmap[:,:] < threshold = 0

class HeatMapAccumulator:

    max_hetamaps=0
    threshold=0
    heatmap_array = []
    heatmap_sum = None

    def __init__(self, size=10, threshold=5):
        self.max_hetamaps = size
        self.threshold = threshold
        self.heatmap_array = []
        self.heatmap_sum = None


    # bounded to size max_hetamaps
    def add_heatmap (self, heatmap):

        if (len(self.heatmap_array) == 0):
            self.heatmap_sum = np.copy(heatmap)
            self.heatmap_array.append(heatmap)
            return

        if (len(self.heatmap_array) == self.max_hetamaps):
            heat = self.heatmap_array.pop(0)
            self.heatmap_sum = np.subtract(self.heatmap_sum, heat)

        self.heatmap_sum = np.add(self.heatmap_sum, heatmap)
        self.heatmap_array.append(heatmap)


    def get_summed_heat_map_after_threshold (self):
        heatmap = np.copy(self.heatmap_sum)
        return self.apply_threshold(heatmap, self.threshold)


    def apply_threshold(self, heatmap, threshold):
        # Zero out pixels below the threshold
        heatmap[heatmap < threshold] = 0
        # Return thresholded map
        return heatmap
`
---

### Discussion

#### 1. Briefly discuss any problems / issues you faced in your implementation of this project.  Where will your pipeline likely fail?  What could you do to make it more robust?

##### A couple of implementation issues I faced

* Figuring out the differences in the imread format for jpeg and png
* The assembling of the various feature vectors and getting the shape of the input parameters right for HOG and for the classifier.

##### How the pipeline was improved

* On the first submit the video pipeline would detect a lot of the cars on the other lane (not terribly bad in iteself). Adjusting the `x_start_stop` the sliding windows were mostly on the driving lane and avoided this issue.
* I added some `hard negative` iamges from previous pipeline results (suitably scaled to 64x64.
* There were several glitches in and false positives along an empty lane on the perevious pipeline result. Adjusted the number of frames averaged (28) and the threshold (24)
* The pipeline result now is satisfactory with virtually no false positive.